In [ ]:
import os
import json
import numpy as np
from unittest.mock import patch, mock_open

In [ ]:
def loadReportJsons():
    baseDir = r"" # test input directory 
    trainedModelData = []
    for root, dirs, files in os.walk(baseDir):
        for file in files:
            if file.endswith(".json"):
                fullPath = os.path.join(root, file)
                with open(fullPath, 'r') as f:
                    data = json.load(f)
                    trainedModelData.append(data)
    return trainedModelData

# get relevant info from trained model data
def getCriteriaData(trainedModelData, maxEpoch=20):
    finalValAcc = trainedModelData["validation_accuracy"]
    epochsData = trainedModelData.get("epochs_data", [])
    usedEpochs = len(epochsData)
    wasEarlyStop = (usedEpochs < maxEpoch)

    # calc training gradient in last x epochs to get rate of increase in val acc
    def getGradient(epochsData, window=5):
        if len(epochsData) < 20:
            return 0.0
        slicedEpochsData = epochsData[-window:]
        diffs = []
        for i in range(len(slicedEpochsData) - 1):
            accDiff = slicedEpochsData[i+1]["val_accuracy"] - slicedEpochsData[i]["val_accuracy"]
            diffs.append(accDiff)
        return sum(diffs)/len(diffs) if diffs else 0.0

    gradient = getGradient(epochsData, window=5)

    return {
        "finalValAcc": finalValAcc,
        "usedEpochs": usedEpochs,
        "wasEarlyStop": wasEarlyStop,
        "gradient": gradient,
    }
    
def adjustForCriteria(finalAccuracy, wasEarlyStop, gradient):
    adj = finalAccuracy
    if wasEarlyStop:
        adj -= gradient * 5 
    adj += gradient * 5
    return adj

In [ ]:
# -------------------------------------------------------
# Test 1: Load report JSONs
# -------------------------------------------------------
def testLoadReportJsons():
    print("\nRunning test: Load report JSONs")

    mockJsonData = {
        "batch_size": 8,
        "image_resize": 256,
        "learning_rate": 0.01,
        "validation_loss": 4.6,
        "validation_accuracy": 4.1,
        "epochs_data": [
            {"epoch": 1, "train_accuracy": 2.3, "train_loss": 62.8, "val_accuracy": 1.3, "val_loss": 4.6},            
        ]
    }

    fakeJsonStr = json.dumps(mockJsonData)

    mockFiles = [
        (r"", # test input directory
         [], ["report1.json", "report2.txt", "report2.json"])
    ]

    with patch("os.walk", return_value=mockFiles):
        with patch("builtins.open", mock_open(read_data=fakeJsonStr)) as mock_file:
            result = loadReportJsons()
            print("Loaded JSON objects:", result)

            # Assertions
            assert isinstance(result, list), "Result should be a list"
            assert len(result) == 2, "Should load only JSON files"
            assert result[0]["validation_accuracy"] == 0.85, "Incorrect JSON content"
            assert "epochs_data" in result[0], "Missing epochs_data"
            print("Passed")
            
# -------------------------------------------------------
# Test 2: Adjust for criteria - early stop
# -------------------------------------------------------
def testAdjustForCriteria():
    print("\nRunning test: Adjust for criteria - early stop")
    
    result = adjustForCriteria(0.8, True, 0.02)
    expected = 0.8
    
    print(f"Result: {result}, Expected: {expected}")
    assert abs(result - expected) < 1e-6
    print("Passed")

# -------------------------------------------------------
# Test 3: Adjust for criteria - no early stop
# -------------------------------------------------------
def testAdjustForCriteria_noEarlyStop():
    print("\nRunning test: Adjust for criteria - no early stop")
    
    result = adjustForCriteria(0.75, False, 0.03)
    expected = 0.75 + (0.03 * 5)
    
    print(f"Result: {result}, Expected: {expected}")
    assert abs(result - expected) < 1e-6
    print("Passed")
    
# -------------------------------------------------------
# Test 4: Get criteria data - early stop
# -------------------------------------------------------
def testGetCriteriaData_earlyStop():
    print("\n[Running test: Get criteria data - early stop")
    
    sample = {
        "validation_accuracy": 0.7,
        "epochs_data": [{"val_accuracy": acc} for acc in np.linspace(0.6, 0.7, 10)]
    }
    result = getCriteriaData(sample)
    
    print("Result:", result)
    assert result["usedEpochs"] == 10
    assert result["wasEarlyStop"] is True
    assert result["gradient"] == 0.0
    print("Passed")

# -------------------------------------------------------
# Test 5: Get criteria data - no early stop
# -------------------------------------------------------
def testGetCriteriaData_normal():
    print("\nRunning test: Get criteria data - no early stop")
    
    sample = {
        "validation_accuracy": 0.9,
        "epochs_data": [{"val_accuracy": acc} for acc in np.linspace(0.7, 0.9, 20)]
    }
    result = getCriteriaData(sample)
    
    print("Result:", result)
    assert result["usedEpochs"] == 20
    assert result["wasEarlyStop"] is False
    assert result["gradient"] > 0
    print("Passed")
    
# Run all
testLoadReportJsons()
testAdjustForCriteria()
testAdjustForCriteria_noEarlyStop()
testGetCriteriaData_normal()
testGetCriteriaData_earlyStop()


Running test: Load report JSONs
Loaded JSON objects: [{'validation_accuracy': 0.85, 'epochs_data': [{'epoch': 1, 'val_accuracy': 0.7}, {'epoch': 2, 'val_accuracy': 0.75}, {'epoch': 3, 'val_accuracy': 0.78}, {'epoch': 4, 'val_accuracy': 0.82}, {'epoch': 5, 'val_accuracy': 0.85}]}, {'validation_accuracy': 0.8, 'epochs_data': [{'epoch': 1, 'val_accuracy': 0.6}, {'epoch': 2, 'val_accuracy': 0.65}, {'epoch': 3, 'val_accuracy': 0.68}, {'epoch': 4, 'val_accuracy': 0.75}, {'epoch': 5, 'val_accuracy': 0.8}]}]
Passed

Running test: Adjust for criteria - early stop
Result: 0.8, Expected: 0.8
Passed

Running test: Adjust for criteria - no early stop
Result: 0.9, Expected: 0.9
Passed

Running test: Get criteria data - no early stop
Result: {'finalValAcc': 0.9, 'usedEpochs': 20, 'wasEarlyStop': False, 'gradient': 0.01052631578947369}
Passed

[Running test: Get criteria data - early stop
Result: {'finalValAcc': 0.7, 'usedEpochs': 10, 'wasEarlyStop': True, 'gradient': 0.0}
Passed
